# 🚀 Interactive DBOR Encoder/Decoder

Welcome to the Interactive DBOR (Data Binary Object Representation) Encoder/Decoder!

This notebook allows you to:
- **Input any Python value** (integers, strings, bytes, lists, etc.)
- **See the DBOR encoding** in both binary and hexadecimal format
- **Decode it back** to verify round-trip integrity
- **Experiment** with different data types and structures

## About DBOR Conformance Level 2

This implementation supports **conformance level 2** which includes:
- `None` values
- Integers in range {−2⁶³ … 2⁶⁴−1}
- Byte strings
- UTF-8 strings 
- Sequences (lists) of supported values

## How to Use This Notebook

1. Run all cells in order
2. Modify the **user input cell** with your own values
3. Execute the encoding and decoding cells
4. Observe the results and verify round-trip integrity

In [3]:
# Import Required Modules
import sys
import os
from typing import Any

# Add src directory to path to import our DBOR implementation
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

try:
    from encoder import encode
    from decoder import decode, DBORDecodeError
    print("✅ DBOR modules imported successfully!")
    print("📦 Available functions: encode(), decode()")
except ImportError as e:
    print(f"❌ Error importing DBOR modules: {e}")
    print("🔧 Make sure you're running this notebook from the project root directory")
    print("📁 Expected structure: src/encoder.py and src/decoder.py")
    
# Helper function for pretty printing
def display_value_info(value: Any, label: str = "Value"):
    """Display detailed information about a value."""
    print(f"\n{label}:")
    print(f"  Value: {value!r}")
    print(f"  Type: {type(value).__name__}")
    if hasattr(value, '__len__'):
        try:
            print(f"  Length: {len(value)}")
        except:
            pass

✅ DBOR modules imported successfully!
📦 Available functions: encode(), decode()


## 📝 User Input Section

**Instructions:** Modify the `user_input` variable in the cell below with any Python value you want to encode!

### Example Values You Can Try:

**Basic Types:**
- `None`
- `42` (integer)
- `-123` (negative integer)
- `"Hello, World!"` (string)
- `b"binary data"` (bytes)

**Unicode Strings:**
- `"Hello, 世界! 🌍"` (mixed languages and emoji)
- `"café naïve üñîçødë"` (accented characters)

**Large Numbers:**
- `2**63 - 1` (large positive integer)
- `-(2**63)` (large negative integer)

**Lists (Sequences):**
- `[1, 2, 3]` (simple list)
- `[None, 42, "hello", b"world"]` (mixed types)
- `[1, [2, [3, 4]], "nested"]` (nested lists)

**Complex Examples:**
- `{"key": "value"}` ⚠️ (will fail - dicts not supported in level 2)
- `[i for i in range(100)]` (large list)
- `[[None] * 10 for _ in range(5)]` (nested structure)

In [8]:
# 🎯 USER INPUT: Modify this value to test different inputs!
# ========================================================

# Change this line to test different values:

def parse_custom_int(val):
    """Parse integers with custom base notation like '16#117'."""
    if isinstance(val, str) and '#' in val:
        base, num = val.split('#', 1)
        return int(num, int(base))
    return val

# Define your input with values within DBOR level 2 range
user_input = [
    # Basic values
    None,
    123456789,
    -123456789,
    0,
    
    # Integer boundaries (within valid range)
    23,                    # 16#17 - last direct encoding
    24,                    # 16#18 - first extended encoding  
    279,                   # 16#117 - boundary value
    280,                   # 16#118 - boundary value
    65815,                 # 16#10117 - larger boundary
    65816,                 # 16#10118 - larger boundary
    
    # Large but valid integers
    2**32 - 1,             # 4,294,967,295
    2**48 - 1,             # 281,474,976,710,655
    2**63 - 1,             # 9,223,372,036,854,775,807 (max signed 64-bit)
    2**64 - 1,             # 18,446,744,073,709,551,615 (max DBOR level 2)
    
    # Negative integers
    -1,
    -24,                   # First negative extended
    -25,                   # -16#19
    -281,                  # -16#119
    -282,                  # -16#11A
    -(2**63),              # Min signed 64-bit
    
    # Strings
    "",
    "¡Olé!",
    "Hello, 世界! 🌍",
    
    # Long text (shortened for better display)
    "This is a sample of Latin text for testing DBOR encoding of longer strings with various characters and punctuation marks.",
    
    # Sequences
    [],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
    [None, [1, [[], [""]]]],
    
    # Mixed types
    [None, 42, "hello", b"world", [1, 2, 3]]
]

# Display information about the input
display_value_info(user_input, "📥 User Input")

# Additional examples to try (uncomment one):
# user_input = None
# user_input = 123456789
# user_input = 2**63 - 1                    # Large positive (valid)
# user_input = -(2**63)                     # Large negative (valid)
# user_input = "🚀 DBOR is awesome! 🌟"
# user_input = b"\x00\x01\x02\xff\xfe\xfd"
# user_input = list(range(20))
# user_input = [[[[[None]]]]]
# user_input = [i**2 for i in range(10)]

# Examples that will fail (for testing error handling):
# user_input = 2**64                        # Too large (will fail)
# user_input = -(2**63) - 1                 # Too small (will fail)
# user_input = {"dict": "value"}            # Unsupported type (will fail)
# user_input = float('inf')                 # Unsupported type (will fail)

print(f"\n✅ User input ready for encoding!")
print(f"💡 All values are within DBOR level 2 limits: integers in [−2⁶³, 2⁶⁴−1]")


📥 User Input:
  Value: [None, 123456789, -123456789, 0, 23, 24, 279, 280, 65815, 65816, 4294967295, 281474976710655, 9223372036854775807, 18446744073709551615, -1, -24, -25, -281, -282, -9223372036854775808, '', '¡Olé!', 'Hello, 世界! 🌍', 'This is a sample of Latin text for testing DBOR encoding of longer strings with various characters and punctuation marks.', [], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], [None, [1, [[], ['']]]], [None, 42, 'hello', b'world', [1, 2, 3]]]
  Type: list
  Length: 29

✅ User input ready for encoding!
💡 All values are within DBOR level 2 limits: integers in [−2⁶³, 2⁶⁴−1]


## 🔐 Encode User Input

This section encodes your input value using the DBOR encoder.

In [9]:
# 🔐 Encoding the user input
print("🔄 Encoding user input...")
print("=" * 50)

try:
    # Encode the user input
    encoded_bytes = encode(user_input)
    
    # Success!
    print("✅ ENCODING SUCCESSFUL!")
    print(f"📦 Input successfully encoded to DBOR format")
    
    # Store for later use
    encoding_success = True
    encoding_error = None
    
except TypeError as e:
    print("❌ TYPE ERROR:")
    print(f"   {e}")
    print("💡 Hint: Check if your input type is supported by DBOR level 2")
    print("   Supported: None, int, str, bytes, list")
    print("   Not supported: dict, set, float, complex, custom objects")
    
    encoded_bytes = None
    encoding_success = False
    encoding_error = e
    
except OverflowError as e:
    print("❌ OVERFLOW ERROR:")
    print(f"   {e}")
    print("💡 Hint: Integer is outside the supported range {−2⁶³ … 2⁶⁴−1}")
    
    encoded_bytes = None
    encoding_success = False
    encoding_error = e
    
except ValueError as e:
    print("❌ VALUE ERROR:")
    print(f"   {e}")
    print("💡 Hint: Check if your input value is valid")
    
    encoded_bytes = None
    encoding_success = False
    encoding_error = e
    
except Exception as e:
    print("❌ UNEXPECTED ERROR:")
    print(f"   {type(e).__name__}: {e}")
    print("💡 Hint: This might be a bug in the encoder")
    
    encoded_bytes = None
    encoding_success = False
    encoding_error = e

print(f"\nEncoding Status: {'✅ SUCCESS' if encoding_success else '❌ FAILED'}")

🔄 Encoding user input...
✅ ENCODING SUCCESSFUL!
📦 Input successfully encoded to DBOR format

Encoding Status: ✅ SUCCESS


## 📊 Display Encoded Output

If encoding was successful, this section shows the encoded bytes in various formats for inspection.

In [17]:
# 📊 Display the encoded output in multiple formats
print("📊 ENCODED OUTPUT ANALYSIS")
print("=" * 50)

if encoding_success and encoded_bytes is not None:
    # Basic information
    print(f"✅ Encoding successful!")
    print(f"📏 Size: {len(encoded_bytes)} bytes")
    print()
    
    # Raw bytes representation
    print("🔢 Raw Bytes:")
    print(f"   {encoded_bytes!r}")
    print()
    
    # Formatted hex (grouped by bytes) - PRIMARY DISPLAY
    hex_string = encoded_bytes.hex()
    hex_formatted = ' '.join([hex_string[i:i+2] for i in range(0, len(hex_string), 2)])
    print("🔍 DBOR Hex Encoding:")
    print(f"   {hex_formatted.upper()}")
    print()
    
    # Compact hexadecimal representation (secondary)
    print("📝 Compact Hex:")
    print(f"   {hex_string.upper()}")
    print()
    
    # Binary representation (first few bytes only if long)
    if len(encoded_bytes) <= 8:
        print("💾 Binary Representation:")
        for i, byte in enumerate(encoded_bytes):
            binary = format(byte, '08b')
            print(f"   Byte {i}: {byte:3d} (0x{byte:02X}) = {binary}")
        print()
    
    # DBOR structure analysis
    if len(encoded_bytes) > 0:
        first_byte = encoded_bytes[0]
        if first_byte == 0xFF:
            print("🔍 DBOR Analysis:")
            print("   Type: None value (special encoding 0xFF)")
        else:
            header = (first_byte >> 5) & 0x7
            payload = first_byte & 0x1F
            
            header_names = {
                0: 'Positive Integer',
                1: 'Negative Integer', 
                2: 'Byte String',
                3: 'UTF-8 String',
                4: 'Sequence'
            }
            
            print("🔍 DBOR Structure Analysis:")
            print(f"   Header: {header} ({header_names.get(header, 'Unknown')})")
            print(f"   Payload: {payload}")
            
            if payload <= 23:
                print(f"   Encoding: Direct (value = {payload})")
            else:
                print(f"   Encoding: Extended ({payload - 23} additional bytes)")
    
else:
    print("❌ No encoded output to display (encoding failed)")
    if encoding_error:
        print(f"   Error: {encoding_error}")

print(f"\n{'='*50}")

📊 ENCODED OUTPUT ANALYSIS
✅ Encoding successful!
📏 Size: 311 bytes

🔢 Raw Bytes:
   b'\x99\x1c\x00\xff\x1b\xfd\xcbZ\x06;\xfc\xcbZ\x06\x00\x17\x18\x00\x18\xff\x19\x00\x00\x19\xff\xff\x1a\x00\x00\x00\x1b\xe7\xfe\xfe\xfe\x1d\xe7\xfe\xfe\xfe\xfe\xfe\x1f\xe7\xfe\xfe\xfe\xfe\xfe\xfe~\x1f\xe7\xfe\xfe\xfe\xfe\xfe\xfe\xfe 78\x009\x00\x009\x01\x00?\xe7\xfe\xfe\xfe\xfe\xfe\xfe~`g\xc2\xa1Ol\xc3\xa9!sHello, \xe4\xb8\x96\xe7\x95\x8c! \xf0\x9f\x8c\x8dxaThis is a sample of Latin text for testing DBOR encoding of longer strings with various characters and punctuation marks.\x80\x97\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x98\x01\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x00\x87\xff\x85\x01\x83\x80\x81`\x93\xff\x18\x12ehelloEworld\x83\x01\x02\x03'

🔍 DBOR Hex Encoding:
   99 1C 00 FF 1B FD CB 5A 06 3B FC CB 5A 06 00 17 18 00 18 FF 19 00 00 19 FF FF 1A 00 00 00 1B E7 FE FE FE 1D E7 FE FE FE FE FE 1F E7 FE FE FE 

## 🔓 Decode Encoded Output

This section decodes the DBOR bytes back to Python objects to verify round-trip integrity.

In [11]:
# 🔓 Decoding the DBOR bytes back to Python objects
print("🔄 Decoding DBOR bytes...")
print("=" * 50)

if encoding_success and encoded_bytes is not None:
    try:
        # Decode the bytes
        decoded_value = decode(encoded_bytes)
        
        # Success!
        print("✅ DECODING SUCCESSFUL!")
        print(f"📦 DBOR bytes successfully decoded back to Python object")
        
        # Store for later analysis
        decoding_success = True
        decoding_error = None
        
    except DBORDecodeError as e:
        print("❌ DBOR DECODE ERROR:")
        print(f"   {e}")
        print("💡 Hint: The encoded bytes may be corrupted or invalid")
        
        decoded_value = None
        decoding_success = False
        decoding_error = e
        
    except Exception as e:
        print("❌ UNEXPECTED DECODING ERROR:")
        print(f"   {type(e).__name__}: {e}")
        print("💡 Hint: This might be a bug in the decoder")
        
        decoded_value = None
        decoding_success = False
        decoding_error = e
        
else:
    print("⚠️  Cannot decode: encoding was not successful")
    decoded_value = None
    decoding_success = False
    decoding_error = "Encoding failed - no bytes to decode"

print(f"\nDecoding Status: {'✅ SUCCESS' if decoding_success else '❌ FAILED'}")

🔄 Decoding DBOR bytes...
✅ DECODING SUCCESSFUL!
📦 DBOR bytes successfully decoded back to Python object

Decoding Status: ✅ SUCCESS


## 🎯 Display Decoded Output & Round-Trip Verification

This final section shows the decoded value and verifies that the round-trip encoding/decoding process preserved the original data perfectly.

In [16]:
# 🎯 Final Results: Decoded Output and Round-Trip Verification
print("🎯 FINAL RESULTS & ROUND-TRIP VERIFICATION")
print("=" * 60)

if encoding_success and decoding_success:
    # Display decoded value
    display_value_info(decoded_value, "📤 Decoded Output")
    
    # Round-trip verification
    print("\n🔄 ROUND-TRIP VERIFICATION:")
    print("-" * 30)
    
    # Check if values are equal
    round_trip_success = (decoded_value == user_input)
    
    print(f"Original Input:  {user_input!r}")
    print(f"Decoded Output:  {decoded_value!r}")
    print(f"Values Equal:    {'✅ YES' if round_trip_success else '❌ NO'}")
    
    # Type verification
    original_type = type(user_input).__name__
    decoded_type = type(decoded_value).__name__
    type_match = (original_type == decoded_type)
    
    print(f"Original Type:   {original_type}")
    print(f"Decoded Type:    {decoded_type}")
    print(f"Types Match:     {'✅ YES' if type_match else '❌ NO'}")
    
    # Overall success
    overall_success = round_trip_success and type_match
    
    print(f"\n{'='*60}")
    if overall_success:
        print("🎉 ROUND-TRIP SUCCESS!")
        print("✅ Perfect data integrity maintained")
        print("✅ decode(encode(input)) == input")
        print("🚀 DBOR encoding/decoding works correctly!")
    else:
        print("⚠️  ROUND-TRIP ISSUES DETECTED")
        if not round_trip_success:
            print("❌ Values don't match after round-trip")
        if not type_match:
            print("❌ Types don't match after round-trip")
        print("🔧 This might indicate a bug in the implementation")
    
    # Encoding efficiency info
    original_size_estimate = len(str(user_input).encode('utf-8'))
    encoded_size = len(encoded_bytes)
    efficiency = encoded_size / original_size_estimate if original_size_estimate > 0 else 0
    
    print(f"\n📊 ENCODING EFFICIENCY:")
    print(f"   Estimated original size: ~{original_size_estimate} bytes")
    print(f"   DBOR encoded size: {encoded_size} bytes")
    print(f"   Compression ratio: {efficiency:.2f}")
    
else:
    print("❌ CANNOT PERFORM ROUND-TRIP VERIFICATION")
    print("   Reason: Encoding or decoding failed")
    
    if not encoding_success:
        print(f"   Encoding error: {encoding_error}")
    if not decoding_success:
        print(f"   Decoding error: {decoding_error}")

print(f"\n{'='*60}")
print("📝 SUMMARY:")
print(f"   Encoding: {'✅ SUCCESS' if encoding_success else '❌ FAILED'}")
print(f"   Decoding: {'✅ SUCCESS' if decoding_success else '❌ FAILED'}")
if encoding_success and decoding_success:
    print(f"   Round-trip: {'✅ SUCCESS' if round_trip_success else '❌ FAILED'}")
print(f"{'='*60}")

🎯 FINAL RESULTS & ROUND-TRIP VERIFICATION

📤 Decoded Output:
  Value: [None, 123456789, -123456789, 0, 23, 24, 279, 280, 65815, 65816, 4294967295, 281474976710655, 9223372036854775807, 18446744073709551615, -1, -24, -25, -281, -282, -9223372036854775808, '', '¡Olé!', 'Hello, 世界! 🌍', 'This is a sample of Latin text for testing DBOR encoding of longer strings with various characters and punctuation marks.', [], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], [None, [1, [[], ['']]]], [None, 42, 'hello', b'world', [1, 2, 3]]]
  Type: list
  Length: 29

🔄 ROUND-TRIP VERIFICATION:
------------------------------
Original Input:  [None, 123456789, -123456789, 0, 23, 24, 279, 280, 65815, 65816, 4294967295, 281474976710655, 9223372036854775807, 18446744073709551615, -1, -24, -25, -281, -282, -9223372036854775808, '', '¡Olé!', 'Hello, 世界! 🌍', 'This is a sample of Latin tex

## 🎮 Experimental Playground

Want to try more examples? Use the cells below to experiment with different values quickly!

In [15]:
# 🎮 Quick Test Function - Try any value instantly!
def quick_test(value):
    """Quickly test encoding/decoding of any value."""
    print(f"🧪 Testing: {value!r}")
    print("-" * 40)
    
    try:
        # Encode
        encoded = encode(value)
        # Format hex with spaces between bytes
        hex_string = encoded.hex()
        hex_formatted = ' '.join([hex_string[i:i+2] for i in range(0, len(hex_string), 2)])
        print(f"✅ Encoded: {hex_formatted.upper()}")
        print(f"📏 Size: {len(encoded)} bytes")
        
        # Decode
        decoded = decode(encoded)
        print(f"✅ Decoded: {decoded!r}")
        
        # Verify
        success = (decoded == value)
        print(f"🔄 Round-trip: {'✅ SUCCESS' if success else '❌ FAILED'}")
        
        return success
        
    except Exception as e:
        print(f"❌ Error: {type(e).__name__}: {e}")
        return False

# Test some interesting examples
print("🚀 QUICK EXAMPLES:")
print("=" * 50)

test_values = [
    None,
    42,
    -123,
    "Hello! 👋",
    b"binary",
    [1, 2, 3],
    [None, "mixed", b"types"]
]

for val in test_values:
    quick_test(val)
    print()

🚀 QUICK EXAMPLES:
🧪 Testing: None
----------------------------------------
✅ Encoded: FF
📏 Size: 1 bytes
✅ Decoded: None
🔄 Round-trip: ✅ SUCCESS

🧪 Testing: 42
----------------------------------------
✅ Encoded: 18 12
📏 Size: 2 bytes
✅ Decoded: 42
🔄 Round-trip: ✅ SUCCESS

🧪 Testing: -123
----------------------------------------
✅ Encoded: 38 62
📏 Size: 2 bytes
✅ Decoded: -123
🔄 Round-trip: ✅ SUCCESS

🧪 Testing: 'Hello! 👋'
----------------------------------------
✅ Encoded: 6B 48 65 6C 6C 6F 21 20 F0 9F 91 8B
📏 Size: 12 bytes
✅ Decoded: 'Hello! 👋'
🔄 Round-trip: ✅ SUCCESS

🧪 Testing: b'binary'
----------------------------------------
✅ Encoded: 46 62 69 6E 61 72 79
📏 Size: 7 bytes
✅ Decoded: b'binary'
🔄 Round-trip: ✅ SUCCESS

🧪 Testing: [1, 2, 3]
----------------------------------------
✅ Encoded: 83 01 02 03
📏 Size: 4 bytes
✅ Decoded: [1, 2, 3]
🔄 Round-trip: ✅ SUCCESS

🧪 Testing: [None, 'mixed', b'types']
----------------------------------------
✅ Encoded: 8D FF 65 6D 69 78 65 64 45 74 

In [18]:
# 🔬 Custom Experiments - Add your own test values here!

# Try these examples or add your own:
my_test_values = [
    # Add your own test values here:
    2**63 - 1,           # Large positive integer
    -(2**63),            # Large negative integer  
    "🚀🌟💯🎉✨",         # Emoji string
    list(range(50)),     # Large sequence
    [[[None]]],          # Nested structure
    b"\x00\x01\x02\xff", # Binary data
    
    # Uncomment to test error cases:
    # 2**64,             # Too large integer (will fail)
    # {"dict": "value"}, # Unsupported type (will fail)
]

print("🔬 YOUR CUSTOM EXPERIMENTS:")
print("=" * 50)

for i, test_val in enumerate(my_test_values, 1):
    print(f"\n{i}. ", end="")
    quick_test(test_val)

print("\n" + "=" * 50)
print("💡 TIP: Modify the 'my_test_values' list above to test your own values!")
print("🎯 Remember: DBOR Level 2 supports None, int, str, bytes, and lists only")

🔬 YOUR CUSTOM EXPERIMENTS:

1. 🧪 Testing: 9223372036854775807
----------------------------------------
✅ Encoded: 1F E7 FE FE FE FE FE FE 7E
📏 Size: 9 bytes
✅ Decoded: 9223372036854775807
🔄 Round-trip: ✅ SUCCESS

2. 🧪 Testing: -9223372036854775808
----------------------------------------
✅ Encoded: 3F E7 FE FE FE FE FE FE 7E
📏 Size: 9 bytes
✅ Decoded: -9223372036854775808
🔄 Round-trip: ✅ SUCCESS

3. 🧪 Testing: '🚀🌟💯🎉✨'
----------------------------------------
✅ Encoded: 73 F0 9F 9A 80 F0 9F 8C 9F F0 9F 92 AF F0 9F 8E 89 E2 9C A8
📏 Size: 20 bytes
✅ Decoded: '🚀🌟💯🎉✨'
🔄 Round-trip: ✅ SUCCESS

4. 🧪 Testing: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
----------------------------------------
✅ Encoded: 98 34 00 01 02 03 04 05 06 07 08 09 0A 0B 0C 0D 0E 0F 10 11 12 13 14 15 16 17 18 00 18 01 18 02 18 03 18 04 18 05 18 06 18 07 18 08 18 09 18 0A 18 0

## 🎉 Conclusion

Congratulations! You've successfully used the Interactive DBOR Encoder/Decoder.

### What You've Learned:
- How to encode Python values to DBOR format
- How to decode DBOR bytes back to Python objects  
- How to verify round-trip integrity
- How DBOR represents different data types internally

### Key Features of DBOR Level 2:
- **None values**: Encoded as single byte `0xFF`
- **Integers**: Direct encoding (0-23) or extended encoding for larger values
- **Strings**: UTF-8 encoded with length prefix  
- **Bytes**: Raw binary data with length prefix
- **Sequences**: Lists of any supported types with length prefix

### Usage Tips:
1. **Modify the user input cell** to test different values
2. **Run cells in order** for proper execution
3. **Check the round-trip verification** to ensure data integrity
4. **Use the experimental playground** for quick testing
5. **Try edge cases** like very large numbers or deep nesting

### Supported Types (Level 2):
✅ `None`  
✅ `int` (in range −2⁶³ to 2⁶⁴−1)  
✅ `str` (UTF-8 strings)  
✅ `bytes` (binary data)  
✅ `list` (sequences of supported types)  

❌ `dict`, `set`, `float`, `complex` (not supported in level 2)

### Next Steps:
- Explore the project's comprehensive test suite
- Check out the analysis tools in the `tools/` directory
- Read the full documentation in `README.md`
- Try building your own DBOR applications!

**Happy encoding! 🚀**

## 🗃️ Export Test Results to CSV

Export all your test results to a CSV file for further analysis or documentation. The CSV will include:
- Original input value (as string representation)
- DBOR encoded bytes (as hex string)
- Decoded output value (as string representation)
- Round-trip success status
- Data type information
- Encoding efficiency metrics

In [ ]:
import csv
import json
from datetime import datetime

def export_to_csv(filename=None):
    """Export test results to CSV file."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"dbor_test_results_{timestamp}.csv"
    
    # Collect all test results
    test_results = []
    
    # Add the current user input test if available
    if 'user_input' in globals() and 'encoded_bytes' in globals():
        test_results.append({
            'input_value': str(user_input),
            'input_type': type(user_input).__name__,
            'encoded_hex': encoded_bytes.hex() if 'encoded_bytes' in globals() else '',
            'encoded_hex_formatted': ' '.join(f'{b:02x}' for b in encoded_bytes) if 'encoded_bytes' in globals() else '',
            'decoded_value': str(decoded_value) if 'decoded_value' in globals() else '',
            'decoded_type': type(decoded_value).__name__ if 'decoded_value' in globals() else '',
            'round_trip_success': round_trip_success if 'round_trip_success' in globals() else False,
            'encoding_success': encoding_success if 'encoding_success' in globals() else False,
            'decoding_success': decoding_success if 'decoding_success' in globals() else False,
            'encoded_size_bytes': len(encoded_bytes) if 'encoded_bytes' in globals() else 0,
            'efficiency_ratio': efficiency if 'efficiency' in globals() else 0.0,
            'test_category': 'user_input'
        })
    
    # Add quick test results if available
    if 'test_values' in globals():
        for i, test_val in enumerate(test_values):
            try:
                encoded = encode(test_val)
                decoded = decode(encoded)
                success = decoded == test_val
                
                # Efficiency ratio: encoded size divided by original UTF-8 size (for strings), else 1.0
                if isinstance(test_val, str):
                    original_size = len(test_val.encode('utf-8'))
                    efficiency_ratio = len(encoded) / original_size if original_size > 0 else 0.0
                else:
                    efficiency_ratio = 1.0

                test_results.append({
                    'input_value': str(test_val),
                    'input_type': type(test_val).__name__,
                    'encoded_hex': encoded.hex(),
                    'encoded_hex_formatted': ' '.join(f'{b:02x}' for b in encoded),
                    'decoded_value': str(decoded),
                    'decoded_type': type(decoded).__name__,
                    'round_trip_success': success,
                    'encoding_success': True,
                    'decoding_success': True,
                    'encoded_size_bytes': len(encoded),
                    'efficiency_ratio': efficiency_ratio,
                    'test_category': 'quick_test'
                })
            except Exception as e:
                test_results.append({
                    'input_value': str(test_val),
                    'input_type': type(test_val).__name__,
                    'encoded_hex': '',
                    'encoded_hex_formatted': '',
                    'decoded_value': '',
                    'decoded_type': '',
                    'round_trip_success': False,
                    'encoding_success': False,
                    'decoding_success': False,
                    'encoded_size_bytes': 0,
                    'efficiency_ratio': 0.0,
                    'test_category': 'quick_test',
                    'error': str(e)
                })
    
    # Add custom experiment results if available
    if 'my_test_values' in globals():
        for i, test_val in enumerate(my_test_values):
            try:
                encoded = encode(test_val)
                decoded = decode(encoded)
                success = decoded == test_val
                
                test_results.append({
                    'input_value': str(test_val),
                    'input_type': type(test_val).__name__,
                    'encoded_hex': encoded.hex(),
                    'encoded_hex_formatted': ' '.join(f'{b:02x}' for b in encoded),
                    'decoded_value': str(decoded),
                    'decoded_type': type(decoded).__name__,
                    'round_trip_success': success,
                    'encoding_success': True,
                    'decoding_success': True,
                    'encoded_size_bytes': len(encoded),
                    'efficiency_ratio': len(encoded) / len(str(test_val).encode('utf-8')) if isinstance(test_val, str) else 1.0,
                    'test_category': 'custom_experiment'
                })
            except Exception as e:
                test_results.append({
                    'input_value': str(test_val),
                    'input_type': type(test_val).__name__,
                    'encoded_hex': '',
                    'encoded_hex_formatted': '',
                    'decoded_value': '',
                    'decoded_type': '',
                    'round_trip_success': False,
                    'encoding_success': False,
                    'decoding_success': False,
                    'encoded_size_bytes': 0,
                    'efficiency_ratio': 0.0,
                    'test_category': 'custom_experiment',
                    'error': str(e)
                })
    
    # Write to CSV
    if test_results:
        fieldnames = [
            'test_category', 'input_value', 'input_type', 
            'encoded_hex', 'encoded_hex_formatted', 'encoded_size_bytes',
            'decoded_value', 'decoded_type', 
            'round_trip_success', 'encoding_success', 'decoding_success',
            'efficiency_ratio', 'error'
        ]
        
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for result in test_results:
                # Ensure all fields exist
                for field in fieldnames:
                    if field not in result:
                        result[field] = ''
                writer.writerow(result)
        
        print(f"✅ Exported {len(test_results)} test results to: {filename}")
        print(f"📊 Summary:")
        print(f"   - Total tests: {len(test_results)}")
        print(f"   - Successful round-trips: {sum(1 for r in test_results if r.get('round_trip_success', False))}")
        print(f"   - Failed tests: {sum(1 for r in test_results if not r.get('round_trip_success', False))}")
        
        # Show categories
        categories = {}
        for result in test_results:
            cat = result.get('test_category', 'unknown')
            categories[cat] = categories.get(cat, 0) + 1
        
        print(f"   - By category: {categories}")
        
        return filename
    else:
        print("⚠️ No test results available to export. Run some tests first!")
        return None

# Export current results
export_filename = export_to_csv()

if export_filename:
    print(f"\n📁 You can find your exported results at: {export_filename}")
    print(f"💡 Tip: You can also call export_to_csv('my_custom_name.csv') to specify a filename.")

✅ Exported 14 test results to: dbor_test_results_20250625_132623.csv
📊 Summary:
   - Total tests: 14
   - Successful round-trips: 14
   - Failed tests: 0
   - By category: {'user_input': 1, 'quick_test': 7, 'custom_experiment': 6}

📁 You can find your exported results at: dbor_test_results_20250625_132623.csv
💡 Tip: You can also call export_to_csv('my_custom_name.csv') to specify a filename.


In [20]:
# 📋 User Input Element-by-Element CSV Export
def export_user_input_elements_csv(filename=None):
    """Export each element of user_input as a separate row in CSV for easy comparison."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"dbor_user_input_elements_{timestamp}.csv"
    
    if 'user_input' not in globals():
        print("⚠️ No user_input variable found. Please run the user input cell first!")
        return None
    
    # Prepare to collect individual element results
    element_results = []
    
    def process_element(element, index, path=""):
        """Process a single element and add it to results."""
        try:
            # Encode the individual element
            encoded = encode(element)
            
            # Decode it back
            decoded = decode(encoded)
            
            # Check round-trip success
            success = (decoded == element and type(decoded) == type(element))
            
            # Format hex output
            hex_compact = encoded.hex()
            hex_formatted = ' '.join(f'{b:02x}' for b in encoded)
            
            # Calculate efficiency for strings
            if isinstance(element, str):
                original_size = len(element.encode('utf-8'))
                efficiency_ratio = len(encoded) / original_size if original_size > 0 else 0.0
            else:
                efficiency_ratio = 1.0
            
            # Add to results
            element_results.append({
                'element_index': index,
                'element_path': path,
                'input_value': str(element),
                'input_repr': repr(element),
                'input_type': type(element).__name__,
                'encoded_hex_compact': hex_compact.upper(),
                'encoded_hex_formatted': hex_formatted.upper(),
                'encoded_size_bytes': len(encoded),
                'decoded_value': str(decoded),
                'decoded_repr': repr(decoded),
                'decoded_type': type(decoded).__name__,
                'round_trip_success': success,
                'values_equal': (decoded == element),
                'types_equal': (type(decoded) == type(element)),
                'encoding_success': True,
                'decoding_success': True,
                'efficiency_ratio': round(efficiency_ratio, 4),
                'error': ''
            })
            
        except Exception as e:
            # Handle encoding/decoding errors
            element_results.append({
                'element_index': index,
                'element_path': path,
                'input_value': str(element),
                'input_repr': repr(element),
                'input_type': type(element).__name__,
                'encoded_hex_compact': '',
                'encoded_hex_formatted': '',
                'encoded_size_bytes': 0,
                'decoded_value': '',
                'decoded_repr': '',
                'decoded_type': '',
                'round_trip_success': False,
                'values_equal': False,
                'types_equal': False,
                'encoding_success': False,
                'decoding_success': False,
                'efficiency_ratio': 0.0,
                'error': f"{type(e).__name__}: {e}"
            })
    
    # Process the main user_input first
    process_element(user_input, 0, "root")
    
    # If user_input is a list, process each element individually
    if isinstance(user_input, list):
        for i, element in enumerate(user_input):
            process_element(element, i + 1, f"[{i}]")
    
    # Write results to CSV
    if element_results:
        fieldnames = [
            'element_index', 'element_path', 'input_value', 'input_repr', 'input_type',
            'encoded_hex_compact', 'encoded_hex_formatted', 'encoded_size_bytes',
            'decoded_value', 'decoded_repr', 'decoded_type',
            'round_trip_success', 'values_equal', 'types_equal',
            'encoding_success', 'decoding_success', 'efficiency_ratio', 'error'
        ]
        
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(element_results)
        
        print(f"✅ Exported {len(element_results)} elements to: {filename}")
        print(f"📊 Element-by-Element Summary:")
        print(f"   - Total elements processed: {len(element_results)}")
        print(f"   - Successful round-trips: {sum(1 for r in element_results if r['round_trip_success'])}")
        print(f"   - Failed round-trips: {sum(1 for r in element_results if not r['round_trip_success'])}")
        
        # Show type breakdown
        type_counts = {}
        for result in element_results:
            input_type = result['input_type']
            type_counts[input_type] = type_counts.get(input_type, 0) + 1
        
        print(f"   - By type: {type_counts}")
        
        # Show encoding size statistics
        sizes = [r['encoded_size_bytes'] for r in element_results if r['encoded_size_bytes'] > 0]
        if sizes:
            print(f"   - Encoding sizes: min={min(sizes)}, max={max(sizes)}, avg={sum(sizes)/len(sizes):.1f} bytes")
        
        return filename
    else:
        print("⚠️ No elements to export!")
        return None

# Create the element-by-element export
print("🔄 Creating element-by-element CSV export...")
print("=" * 60)

element_export_filename = export_user_input_elements_csv()

if element_export_filename:
    print(f"\n📁 Element-by-element results saved to: {element_export_filename}")
    print(f"💡 This CSV contains one row per element, perfect for side-by-side comparison!")
    print(f"📋 Columns include: input, encoded hex, decoded output, success status, and more")

🔄 Creating element-by-element CSV export...
✅ Exported 30 elements to: dbor_user_input_elements_20250625_133248.csv
📊 Element-by-Element Summary:
   - Total elements processed: 30
   - Successful round-trips: 30
   - Failed round-trips: 0
   - By type: {'list': 6, 'NoneType': 1, 'int': 19, 'str': 4}
   - Encoding sizes: min=1, max=311, avg=20.6 bytes

📁 Element-by-element results saved to: dbor_user_input_elements_20250625_133248.csv
💡 This CSV contains one row per element, perfect for side-by-side comparison!
📋 Columns include: input, encoded hex, decoded output, success status, and more
